# LLM Text Auto-Completion


## Overview

This notebook contains a mock up of an autocomplete system for legal services queries. It uses various components such as previous completions, domain knowledge, and user input to generate "auto-completions".

### Key Components
**Dynamic BAGs**: Dynamic, big ass prompts with general domain knowledge, few-shot examples, and detailed instructions.
**Adaptive Learning**: LLM auto-completes self-improve with each (confirmed correct) interaction, becoming more accurate over time.

___

## Code Breakdown

In [1]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from IPython.display import Markdown, display, HTML

### Constants
```python
PROMPT_ROOT_DIR = "prompts"
TOPIC_DIR = "LSS"
TOPIC = "Legal Services Department Questions"
```

In [2]:
PROMPT_ROOT_DIR = "prompts"
TOPIC_DIR = "LSS"
TOPIC = "Legal Services Department Questions"

### Load Prompt Template `Instructions`

In [3]:
prompt = open(f"{PROMPT_ROOT_DIR}/prompt.txt", "r").read()
print(prompt)

# You're a superhuman autocomplete system that provides auto-completions for your users.
You take the TOPIC, the PREVIOUS_COMPLETIONS, DOMAIN_KNOWLEDGE and you generate a list of the most likely auto completions for your users based on their INPUT_VALUE.

You closely follow GENERATION_RULES to provide the best possible completions.

## GENERATION_RULES
- If the users INPUT_VALUE exists within their PREVIOUS_COMPLETIONS, prefer that completion. Always prefer the completion with the highest hits.
- If the users INPUT_VALUE does NOT exist in PREVIOUS_COMPLETIONS, derive completions from DOMAIN_KNOWLEDGE.
- If the users INPUT_VALUE isn't in PREVIOUS_COMPLETIONS and doesn't have a completion in DOMAIN_KNOWLEDGE, generate a new, concise completion based on your own knowledge of the TOPIC.
- Return the list of completions as JSON in this format {completions: ["...", "...", "..."]}
- Provide completions that fully complete the users sentence.
- Your completions should be the remaining words in

### Function: `build_omni_complete_prompt`
* Get JSON few-shot examples sorted by past experience 'hits'

In [4]:
def build_omni_complete_prompt(input_value: str, topic: str = "Legal Services Department Questions", topic_dir: str = "LSS") -> str:
    """
    Constructs a complete prompt for the Omni autocomplete system by incorporating various components.

    Args:
        input_value (str): The user's input question or query.
        topic (str): The specific topic under which the prompt is categorized.
        topic_dir (str): The directory associated with the topic.

    Returns:
        str: A fully constructed prompt ready for use in the autocomplete system.

    """
    prompt = open(f"{PROMPT_ROOT_DIR}/prompt.txt", "r").read()

    previous_completions = open(
        f"{PROMPT_ROOT_DIR}/knowledge_bases/{topic_dir}/previous_completions.json", "r"
    ).read()

    domain_knowledge = open(
        f"{PROMPT_ROOT_DIR}/knowledge_bases/{topic_dir}/domain_knowledge.txt", "r"
    ).read()

    prompt = prompt.replace("{{topic}}", topic)
    prompt = prompt.replace("{{previous_completions}}", previous_completions)
    prompt = prompt.replace("{{domain_knowledge}}", domain_knowledge)
    prompt = prompt.replace("{{input_value}}", input_value)
    return prompt

- **Purpose**: Constructs a complete prompt by incorporating the topic, previous completions, domain knowledge, and user input.

In [7]:
prompt_example = build_omni_complete_prompt("who can help me with a question about pollution damage?")
print(prompt_example)

# You're a superhuman autocomplete system that provides auto-completions for your users.
You take the TOPIC, the PREVIOUS_COMPLETIONS, DOMAIN_KNOWLEDGE and you generate a list of the most likely auto completions for your users based on their INPUT_VALUE.

You closely follow GENERATION_RULES to provide the best possible completions.

## GENERATION_RULES
- If the users INPUT_VALUE exists within their PREVIOUS_COMPLETIONS, prefer that completion. Always prefer the completion with the highest hits.
- If the users INPUT_VALUE does NOT exist in PREVIOUS_COMPLETIONS, derive completions from DOMAIN_KNOWLEDGE.
- If the users INPUT_VALUE isn't in PREVIOUS_COMPLETIONS and doesn't have a completion in DOMAIN_KNOWLEDGE, generate a new, concise completion based on your own knowledge of the TOPIC.
- Return the list of completions as JSON in this format {completions: ["...", "...", "..."]}
- Provide completions that fully complete the users sentence.
- Your completions should be the remaining words in

### If tokens are a concern we can truncate the examples

In [8]:
import tiktoken

def count_tokens(string: str, model: str = "gpt-3.5-turbo") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

- **Example**: Demonstrates how to use the `truncate_json_by_tokens` function.

In [9]:
count_tokens(prompt_example)

2736

In [10]:
import json
from typing import Any, Dict, List

def truncate_json_by_tokens(json_string: str, threshold: int, model: str = "gpt-3.5-turbo") -> List[Dict[str, Any]]:
    """
    Truncates a JSON string into a list of dictionaries based on a token threshold using a specified model.

    Args:
        json_string (str): The JSON string to be truncated.
        threshold (int): The maximum number of tokens allowed in the truncated output.
        model (str): The model used to count tokens, default is "gpt-3.5-turbo".

    Returns:
        List[Dict[str, Any]]: A list of dictionaries representing the truncated JSON data.
    """
    data = json.loads(json_string)
    truncated_data = []
    current_tokens = 0

    for item in data:
        item_string = json.dumps(item, indent=4)
        item_tokens = count_tokens(item_string, model)
        
        if current_tokens + item_tokens > threshold:
            break
        
        truncated_data.append(item)
        current_tokens += item_tokens

    return truncated_data

In [11]:
previous_completions = open(
    f"{PROMPT_ROOT_DIR}/knowledge_bases/LSS/previous_completions.json", "r"
).read()

count_tokens(previous_completions)

1545

In [12]:
truncated_json = truncate_json_by_tokens(previous_completions, 200)
count_tokens(str(truncated_json))

131

### Class: `AutoCompletionEntry`
* This class represents historical data and handles updates

In [14]:
from pydantic import BaseModel
from typing import List

class AutoCompletionEntry(BaseModel):
    input: str
    completions: List[str]
    correct_department: str | None= None
    hits: int = 1
    
    def __str__(self):
        return f"{self.input} {self.completions[0]}"


In [15]:
AutoCompletionEntry(**truncated_json[0])

AutoCompletionEntry(input='i have a case where an employee is disputing', completions=['their termination and i need guidance'], correct_department='Employment Disputes', hits=8)

In [16]:
input = 'i need legal advice on a claim involving'
completion = 'a dangerous product'

new_completion = AutoCompletionEntry(input=input, completions=[completion])

In [17]:
print(new_completion)

i need legal advice on a claim involving a dangerous product


### Function: `increment_or_create_previous_completions`
- Updates or creates new entries in the existing completions list
- Increments hit counts, sorts by hits, and saves back to the file

In [18]:
import json
from typing import List
from pydantic import BaseModel, ValidationError

def increment_or_create_previous_completions(input: str, completion: str, topic_dir: str) -> List[AutoCompletionEntry]:
    """
    Updates the list of previous completions by incrementing the hit count for existing entries
    or creating a new entry if no match is found. The list is then sorted by hits in descending order
    and saved back to the file.

    Args:
        input (str): The user input string to match or add.
        completion (str): The completion string associated with the input.
        topic_dir (str): The directory name under which the completions file is stored.

    Returns:
        List[AutoCompletionEntry]: A list of AutoCompletionEntry objects sorted by hits in descending order.
    """
    previous_completions_file = (
        f"{PROMPT_ROOT_DIR}/knowledge_bases/{topic_dir}/previous_completions.json"
    )

    # Try to read the previous completions file
    try:
        with open(previous_completions_file, "r") as file:
            data = json.load(file)
            previous_completions = [AutoCompletionEntry(**item) for item in data]
    except (FileNotFoundError, json.JSONDecodeError, ValidationError):
        previous_completions = []

    # Check for a matching input and completion
    matching_case = None
    for item in previous_completions:
        if item.input.lower() == input.lower():
            for existing_completion in item.completions:
                if completion.lower() in existing_completion.lower():
                    matching_case = item
                    break

    # Update hits if a match is found, otherwise create a new entry
    if matching_case:
        matching_case.hits += 1
    else:
        new_completion = AutoCompletionEntry(input=input, completions=[completion])
        previous_completions.append(new_completion)

    # Sort completions by hits in descending order
    completions_sorted_by_hits = sorted(
        previous_completions, key=lambda x: x.hits, reverse=True
    )

    # Write the updated completions back to the file
    with open(previous_completions_file, "w") as file:
        json.dump([item.model_dump() for item in completions_sorted_by_hits], file, indent=4)

    return completions_sorted_by_hits

In [19]:
import openai

def prompt_json(prompt: str) -> str:
    """
    Sends a prompt to the OpenAI API and retrieves a JSON-formatted response.

    Args:
        prompt (str): The text prompt to send to the model.

    Returns:
        str: The content of the message from the model's response.
    """
    client = openai.OpenAI()
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        stream=False,
        model="gpt-3.5-turbo",
        response_format={
            "type": "json_object",
        },
    )
    return chat_completion.choices[0].message.content

In [40]:
from typing import Sequence
from pydantic import Field, model_validator, ConfigDict

class AutoCompletions(BaseModel):
    """Auto-completions for a new user query."""
    
    input: str = Field(
        ...,
        description="The user provided INPUT_VALUE.",
    )
    completions: List[str] = Field(
        default_factory=list,
        description="A list of potential completions based on the GENERATION_RULES.",
    )
    correct_department: str = Field(
        ...,
        description="The predicted department based on ALL available information.",
    )
    

In [42]:
import instructor
from openai import OpenAI
from typing import Any

def get_autocompletions(input_data: str) -> AutoCompletions:
    """
    Retrieves auto-completions for a given input using a language model.

    Args:
        input_data (str): The user's input data for which completions are needed.

    Returns:
        AutoCompletion: An object containing the input, completions, and predicted department.
    """
    prompt = build_omni_complete_prompt(
        input_data, topic=TOPIC, topic_dir=TOPIC_DIR
    )
    client = instructor.from_openai(OpenAI())
    
    return client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            model="gpt-4-turbo",
            response_model=AutoCompletions,
        )

In [43]:
input_data = 'I need assistance with a legal issue involving'

autocompletions = get_autocompletions(input_data)

In [44]:
autocompletions.model_dump()

{'input': 'I need assistance with a legal issue involving',
 'completions': ['a legal dispute',
  'an arbitration',
  'a regulatory issue',
  'property damage',
  'a personal injury'],
 'correct_department': 'General Legal Queries'}

In [48]:
def get_autocomplete(input_data: str) -> AutoCompletionEntry:
    """
    Fetches autocompletion entries for a given input using a predefined prompt structure.

    Args:
        input_data (str): The user's input data for which autocompletions are needed.

    Returns:
        AutoCompletionEntry: An object containing the input and its corresponding completions.
    """
    prompt = build_omni_complete_prompt(
        input_data, topic=TOPIC, topic_dir=TOPIC_DIR
    )
    client = instructor.from_openai(OpenAI())
    
    return client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            model="gpt-4-turbo",
            response_model=AutoCompletionEntry,
        )

In [49]:
input_data = 'I have a complex coverage dispute involving'
print(input_data)
autocomplete_object = get_autocomplete(input_data)
print(autocomplete_object.completions[0])
print(str(autocomplete_object))

I have a complex coverage dispute involving
claims or coverage limits
I have a complex coverage dispute involving claims or coverage limits


In [50]:
autocomplete_object.model_dump()

{'input': 'I have a complex coverage dispute involving',
 'completions': ['claims or coverage limits'],
 'correct_department': 'Coverage Team',
 'hits': 1}

In [51]:
updated_completions = increment_or_create_previous_completions(
        autocomplete_object.input, autocomplete_object.completions[0], TOPIC_DIR
    )

In [104]:
import requests

class SearchRequest(BaseModel):
    body: str

def get_autocomplete(user_query: str) -> list[str]:
    input_data = SearchRequest(body=user_query)
    url = "http://127.0.0.1:5000/get-pred"
    
    response = requests.post(url, data=input_data.model_dump_json(), headers={"Content-Type": "application/json"})
    return AutoCompletions(**response.json())

In [111]:
input_text = "I need assistance with a complex liability"
res = get_autocomplete(input_text)

In [112]:
res.model_dump()

{'input': 'I need assistance with a complex liability',
 'completions': ['claim involving a multi-party dispute',
  'case requiring in-depth analysis of contractual obligations'],
 'correct_department': 'Insurance Defense Team'}